In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
from gensim.models import word2vec
import pickle

vector_size = 250

In [2]:
'''
csv読み込み
store_idをindexに変更
'''

df = pd.read_csv('../csv/review_wakati.csv')
df = df.drop(['store_id'],axis=1)
df = df.rename(columns={'Unnamed: 0':'store_id'}).set_index('store_id')

In [3]:
'''
pickleで学習済みreview達のvector読み出し
'''

f = open("../pickle/skip_list.txt","rb")
skip_list = pickle.load(f)
f.close()

In [4]:
'''
学習済みモデルの読み出し
'''

model = word2vec.Word2Vec.load('../models/skip_w2v.model')

In [5]:
'''
ここまで準備
'''

'\nここまで準備\n'

In [6]:
def sentence_similarity(vec1, vec2):
    '''
    二つの文章のコサイン類似度を返す
    '''
    return 1 - spatial.distance.cosine(vec1, vec2)

In [10]:
def avg_feature_vectors(words, model):
    feature_vec = np.zeros(vector_size, dtype='float32')
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError:
            pass

    return feature_vec

In [23]:
'''
store_idをkeyに指定文書と各行のreviewのコサイン類似度をスコアとして入れる
'''

result = {}
keyword = avg_feature_vectors(['豚骨', '博多'], model)
for i, row in df.iterrows():
    score = sentence_similarity(keyword, skip_list[i])
    result[i] = score * row.score


/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [24]:
'''
scoreの高い順にsort
'''

score_sorted = sorted(result.items(), key=lambda x:x[1], reverse=True)

In [25]:
'''
score高い10件を書き出し
'''

for t in score_sorted[:10]:
    row = df[df.index == t[0]]
    name, score, station = row.store_name.values[0], row.score.values[0], row.station.values[0]
    print('店名:{}, 食べログスコア:{}, 独自スコア:{:.2f}, 最寄駅:{}'.format(name,score,t[1]*100,station))
    

店名:長浜屋台 やまちゃん 銀座店, 食べログスコア:3.6, 独自スコア:245.02, 最寄駅:東銀座
店名:博多長浜ラーメン 呑龍, 食べログスコア:3.58, 独自スコア:243.47, 最寄駅:汐留
店名:博多ラーメン ばりこて, 食べログスコア:3.73, 独自スコア:241.84, 最寄駅:東高円寺
店名:博多長浜らーめん いっき, 食べログスコア:3.7, 独自スコア:234.34, 最寄駅:谷在家
店名:田中商店 ダイバーシティ東京プラザ店, 食べログスコア:3.66, 独自スコア:232.85, 最寄駅:東京テレポート
店名:博多一瑞亭 三田店, 食べログスコア:3.54, 独自スコア:232.71, 最寄駅:三田
店名:長浜ナンバーワン  東京大岡山店, 食べログスコア:3.61, 独自スコア:230.37, 最寄駅:大岡山
店名:博多らーめん 濱田屋 北千住店, 食べログスコア:3.52, 独自スコア:228.15, 最寄駅:北千住
店名:博多食堂, 食べログスコア:3.51, 独自スコア:228.07, 最寄駅:神泉
店名:博多豚骨ラーメン田原, 食べログスコア:3.53, 独自スコア:227.94, 最寄駅:堀切菖蒲園
